## Practice running the workflow through 3dtiles tiling

- use environment with `viz-staging` and `viz-raster` installed
- after running through these steps in chunks in this notebook, it's a great idea to transfer the code to a script and run as a `tmux` session

In [ ]:
# directory in which to look for data (change this to your needs!)
DATA_DIR = '/home/shares/drp/pointcloud'

#### Imports

In [1]:
# input data import
from pathlib import Path

# py3dtiles
from py3dtiles.tileset.utils import TileContentReader

# interaction with the system
import subprocess

# logging
from datetime import datetime, timedelta
import logging
import os

### Define the logging configuration

This prints logging statements to a file specified by the path in the config. Change the filepath as needed. There will be many logging statements written to that file. It's helpful to ctrl + f for certain logged statements when troubleshooting. For example, if fewer files were staged than expected, you can search for "error" or "failed". If you are debugging a silent error and suspect that the issue has something to do with the order in which input files are processed, you can search for the input filenames to determine which was staged first. In between runs, it's a good idea to delete the log from the past run, rename it, or archive it elsewhere so the next run's log does not append to the same log file. 

In [2]:
DATE_FMT = '%Y-%m-%dT%H:%M:%S'
LOG_FMT = "%(asctime)s:%(levelname)s: %(message)s" # overrides import
L = logging.getLogger('DRPWorkflow')
L.setLevel("INFO")
handler = logging.handlers.WatchedFileHandler(
    os.environ.get("LOGFILE", os.path.expanduser("~/bin/drpworkflow/log/log.log"))) # <- note: should find this dir programatically in the future
formatter = logging.Formatter(fmt=LOG_FMT, datefmt=DATE_FMT)
handler.setFormatter(formatter)
L.addHandler(handler)
L.info("Logging start.")

### Import data

In order to process 1 or 2 files instead of all 3 adjacent files on Wrangle Island, subset the `flist` list of filepaths created below. Estimated times and number of files 

In [3]:
base_dir = Path(DATA_DIR)
vlrcorrect_dir = os.path.join(base_dir, 'vlrcorrect')
archive_dir = os.path.join(base_dir, 'archive')
out_dir = os.path.join(base_dir, '3dtiles')
for d in [vlrcorrect_dir, archive_dir, out_dir]:
    L.info('Creating dir %s' % (d))
    os.makedirs(d, exist_ok=True)
filename = '*.las'
# To define each .las file within each subdir as a string representation with forward slashes, use as_posix()
# ** represents that any subdir string can be present between the base_dir and the filename (not using this because we don't want to include subdirs)
flist = [p.as_posix() for p in base_dir.glob('./' + filename)]
L.info('File list: %s' % (flist))
flist

['/home/shares/drp/pointcloud/Site7.las']

### Use las2las to write new VLR

QT Modeler and other LiDAR processing software sometimes outputs VLR (variable length record) header information that does not adhere exactly to LAS standards. And sometimes, the data abstraction libraries like `PDAL` that undergird the tools we use on the Python side don't play nicely with those headers. Accordingly, sometimes it is necessary to use software that handles malformed headers gracefully like `lastools` to write headers that `PDAL` will accept. Here we use [`las2las`](https://downloads.rapidlasso.de/las2las_README.txt) to rewrite input LAS files with the correct VLR format.

In [4]:
las2lasstart = datetime.now()
L.info('Using las2las to resolve any issues with malformed VLR (variable length record) from QT Modeler... (step 1 of 3)')
i = 0
for f in flist:
    i += 1
    L.info('Processing %s (%s of %s)' % (f, i, len(flist)))
    bn = os.path.basename(f)
    vlrcn = os.path.join(vlrcorrect_dir, bn)
    an = os.path.join(archive_dir, bn)
    subprocess.run([
        '../bin/las2las',
        '-i',
        f,
        '-epsg', '4326',
        '-wgs84',
        '-meter',
        '-target_ecef',
        '-target_epsg', '4326',
        '-o',
        vlrcn
    ])
    # move the file to the archive
    L.info('Archiving to %s' % (an))
    os.replace(src=f, dst=an)
las2lastime = (datetime.now() - las2lasstart).seconds/60
L.info('Finished las2las rewrite (%.1f min)' % (las2lastime))

### Generate new file list with VLR-corrected files

In [5]:
# generate new file list
flist = [p.as_posix() for p in Path(vlrcorrect_dir).glob('./' + filename)]
flist

['/home/shares/drp/pointcloud/vlrcorrect/Site7.las']

### Use py3dtiles to tile point cloud data into web-ready chunks

3dtiles is a format that breaks large datasets up into manageable chunks that can be easily downloaded and displayed at varying zoom levels. This conversion process computes and executes the tiling, then writes outputs.

In [6]:
processstart = datetime.now()
L.info('Starting tiling process for %s file(s) (step 2 of 3)' % (len(flist)))
i = 0
for f in flist:
    i += 1
    tilestart = datetime.now()
    L.info('Processing %s (%s of %s)' % (f, i, len(flist)))
    L.info('Creating tile directory')
    fndir = os.path.join(out_dir, os.path.splitext(os.path.basename(f))[0])
    subprocess.run([
        'py3dtiles',
        'convert',
        f,
        '--out',
        fndir,
        '--overwrite'
    ])
    tiletime = (datetime.now() - tilestart).seconds/60
    L.info('Done (%.1f min)' % (tiletime))
processtime = (datetime.now() - processstart).seconds/60
L.info('Finished tiling (%.1f min)' % (processtime))

 100.0 % in 9 sec [est. time left: 0 sec]]]]]

### Merge outputs of tiling runs on multiple input files

This process takes note of the outputs of various tiling runs (put into subdirectories) and gathers the relative paths of all those outputs into one tileset JSON file.
For example, consider this directory tree:

```
Sites_3dtiles
├── Site4
│   ├── r0.pnts
│   ├── r1.pnts
│   ├── r2.pnts
│   ├── ...
│   ├── r.pnts
│   └── tileset.json
├── Site5
│   ├── r0.pnts
│   ├── r1.pnts
│   ├── r2.pnts
│   ├── ...
│   ├── r.pnts
│   └── tileset.json
└── tileset.json
```
Each subdirectory's `tileset.json` file would have pointers at and metadata for each of the tiles in the subdirectory, and the master `tileset.json` would point at each of those subdirectory tilesets in turn.

The following code is designed to create the master tileset which points to all the various subdirectory tilesets.

In [7]:
L.info('Starting merge process in %s (step 3 of 3)' % (out_dir))
mergestart = datetime.now()
subprocess.run([
    'py3dtiles',
    'merge',
    '--overwrite',
    '--verbose',
    out_dir
])
mergetime = (datetime.now() - mergestart).seconds/60
L.info('Finished merge (%.1f min)' % (mergetime))

Found 1 tilesets to merge
------------------------


## Visualizing the data in Cesium

You can view the output tiles on a Cesium basemap! For steps for how to visualize the tiles with local Cesium, see [documentation here in pdg-info](https://github.com/julietcohen/pdg-info/blob/main/05_displaying-the-tiles.md#option-1-run-cesium-locally).

Here is an example of the `cesium.js` file that ended up working for me (you will need your own access token):


```javascript

function start(){// Your access token can be found at: https://cesium.com/ion/tokens.

  Cesium.Ion.defaultAccessToken = "YOUR-TOKEN-HERE"

  const viewer = new Cesium.Viewer('cesiumContainer');

  const imageryLayers = viewer.imageryLayers;

  var tileset = new Cesium.Cesium3DTileset({
    url: "3dtiles/tileset.json",
    debugShowBoundingVolume: true,
    debugShowContentBoundingVolume: false,
    debugShowGeometricError: false,
    debugWireframe: true
  });

  viewer.scene.primitives.add(tileset);

  window.zoom_to_me = function(){
    viewer.zoomTo(tileset);
  }

  tileset.readyPromise.then(zoom_to_me).otherwise(error => { console.log(error) });
}

start()
```

## More visualization options

The above `cesium.js` code will display the LiDAR data with its RGB values. However if the dataset doesn't have RGB values assigned, the points in the dataset will display black until you colorize them. There's some Cesium [learning material](https://cesium.com/learn/cesiumjs-learn/cesiumjs-3d-tiles-styling/#style-point-clouds) about point cloud stylizing, but here are a couple of other options to consider:

- You can (sort of) [color points in discrete elevation bands](https://sandcastle.cesium.com/#c=tVdrb9s2FP0rhFegMmbLevmh1AnWeq8A3VLMbvdhHgpKom0iEimQlF0v8H/fJSk5cpygXdcECWxe8t5z7pPMFgu0pWRHBLpEjOzQjEhaFe4HI3OWndSsZ5wpTBkRy04P3S0ZQooIAZJ3gm9pRsRFo5gKghX5k4s8W9gjTre3ZIfuqyVbsi3AKZoTSRTgWWBXpoQRtxS0oIpuiXRxljkao8XHfoQ/LqyyYzggVIn8iHzN2R9E8kqkxF0JXryWcPA6c4JJHA01B61QcsrULOdVNt/gjLL1BapNIYSVIqzCinJ2gZSoSK/ZKfAnWlTF6/aBoN49mM9Dd8msi7V7LsQh20N4CiqJqzaEOauKpVrX6VrMwQDN1T4nKMGSZIgztCF0vVFIs0ckz2kpOc3qo4sNlQh+MUO4LAUHRoYJCDK0wbCBVhCvUpCUSi1P8RZSIWt1hHwXbBC0qgSwEQjv8B50TEC0XQMKO8f8QFYgycihTCe7kIivUM7ZmqoqIz2UA7r+1jVKBRcEUcZwWgnIv5HV3iQk5QW55xG46N3N/HpxffP7x9dv5jdv3y9+Mo6hMOgnVKEtzittzBiRkCZgsdvQdKNPMa6QrFYrmlLg13J3xaG0dIxkinNi0Y6hA0I7jsBjiRRHtID4bYk9jgVph2jGi7ICB0oi+jY2x0Q0DnHLbPbuvYl9gW+BrUJ4i2mOE8gnQBjqJrkmM0lFc9UHl6DGBE0AoB2OBtPal43rOhHCQEhIpTUExIGZ2mv1VjPNbK4um7Wb8Irp8p6XkGvi2ly+arRkJVY4JTMsFF8LXEJwQbdptZbUNJIWwBZmzgla15h7xJRbx+kSea73GKYx9gSg4vdwj9juHu1lVCrMUrLgc3vs1KCxELoFXjNTps5DfNut6Bgxm63Lp4aO6dRm7GRkBcNQtmaHdfkCLTs5YWu1cV7cnRX5oYv6cAB9f869PUoQSnnORct4yiGXutcB8a9GiOBr58WdBT6gKYog2no8w8zW+s5LLjBbk5fdZefv3pNaw4dae6h3vvuM1uihVk6LzyGNH+qke8we0dGT9+Qc9L6yxptz9ZeDnbyvjtcLNNTP+V73H0aMQklAlwt9y+jZpVsqpWrvwjVkL54UF0Rgd5XvF/w+tVJRVs/4di0ca8qxB/1gNPYmvjvy42E4mkzC2ot+FMTeOIrc2B/74SQK43ojGoZeHPmuF/lDP479WIvtvcSFHmY1aEOEwmj7PA0A9NxxOJz4oReNh74Xe5NjPD13FMDlB/KRF4aTYBS0toIoHof+MI7C4RB2/cBuNTdlVX4BsBeP41EcRqNJFAVh4I97rc1oBO6PRwA+CoajY4A09mTiRcDYDzxgF7eBbQ9k+hYxvnt1cju9ztS06FVj5QcY41wo/QZwXHegSFHCnUTkIKnSW2joVEo7LfTPd4rzPIGxcXdfawlOb9dCD8oLJNYJdqKgh5o/4HivDe8GeJOY10JUfmqJEy7gduoLeEpU8nTzcAZNGUz5NgEoTkXhturjnK7B14JmWU7OUfuKQzKCE+RmK+FK8eJ09xwaRrK5RlvgK3jS9Xf12Ep4nj1Unw7a8Z5mdItodvnIixClOZYSdlZVns/pP9C9V9MBnD9Tzbl5c92A4zne62Mb/+qtFbquOx3A8nHN2hFQaVhOlb5rr+4dmqqEZ/uWQIvEyVpLsiNdGxMwOdMCCk8K+6Ba7EsipwOVndoaPDD2qPGrX0xBnWnbzaktAQUAOpAbkt4m/BNEMMMKw9uHZY2YQE3a2jTB/Doub/kOfSBrYifLE5xOJSD7Txzhqvi4PUK00nMk+lXMfyPQUMVzky8MynPw/xXa6rnZbwDjObi/gbeqbkj5TYo4aaz9jzq+0f+wDN6ztO5T8m36i2uzX0ALlieTBdat0dOeV/8C)
- Color points by [proximity to a point in xyz space](https://sandcastle.cesium.com/#c=dVTbjts2EP0VwghgCVBp6i4lziKOu0WMul1j7TQvAgKaorNsaNIgKW3dhf+9pC5OnSB6sDzDmXNm5gw1m20kEwYsuWxqsD+DjZIHCDbUUAUWnFMRgKXkzXHPMPgoWEuVZuYMHuVeGkY0WOM9BFuChWDiiwNYcPo3FrWSYKckIZIzYE3wBzaUgSWTBCvOKKxEixVoGX22RG+BoM9gSTVrjvCvzudNSWcupTCYCaqmAXipBLCPLU1Zl620ZbXNfj1mEkUtyyepeL3rYzy/Ehf/TSV6uidbCf+eb2vTqNieMKH3LRXmQx/k9bVBTaig0DbYYu2QBgyoqVmJU2MWxDApvEMj+j/UYfhjrY7VdmwsFRaW9wb0xMjXjbTzvObB02A6JpdPpNCSU8jlF++K4w4vwc/q351PFK7vf9t9Xq5Xy9+/da/NmVNbw1BaTQ92rtqOb3B0TqYNFoRa75RwfDx5o8d79bJ52K52q4c/Py/ebx/WH3f3toiWkthLEhRmZQmTMInLMgyTAERRisIIZqjMwyxMUBqAJC3TJA5hEucoTKOi9H0wA1EKUQCQ+wkh8qd9LZdg7J9Lp/D0yP7xOsOrJmfKuXyuJn4ARpeidWe/ehnrvTioy7V5wzi1kt1K37/iX3f9ofcCGsWv67SS4pFq2ShC4UHJ40I7zWsvzJIo9EG3VwMsHGf7E/CtO/a6IJd1uwWKHa3mLdUQ17U3IA6y3e7gkSrsFs9dEe8qozZMYLc0r2/ox23p5cmTBGaJFSiNR3lSK0WI8sIKhtJo0CdCMM6TvEiSOPYHEaRidq9+pPhAcW1v/YYZ8vQoOfdSS5HlcVygFMVlnuYB+AXBKM8ylIdpXMRlUWQByGBUhJa0yNMEIVSk327pJJjMuzHd9dTv2PEklXG6eBDODD2euL3jerZvyFc7d6K7SzmfjUnzmrWA1W+ryXffj2oC7EJrbU8ODedb9i+tJnfzmY2/SeOya+rBfuk4PruQp/Bu3TshhPOZNX/MMlLyPVb/Q/wP)
- And in the future, Cesium plans on [making elevation accessible to a custom shader](https://github.com/CesiumGS/cesium/issues/9735). There is also a more detailed discussion of this in [CesiumGS/3d-tiles#603](https://github.com/CesiumGS/3d-tiles/issues/603).
